In [164]:
import pandas as pd
from sklearn import preprocessing
import numpy as np
from tqdm.auto import tqdm
import spacy
from spacy import displacy
from spacy.matcher import PhraseMatcher
from tensorflow.keras.preprocessing.text import Tokenizer
from numpy import array
from numpy import asarray
from numpy import zeros
nlp = spacy.load("en_core_web_sm")

In [165]:
from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import *

from tensorflow.keras.layers import Flatten, LSTM, Bidirectional, Embedding
from tensorflow.keras.layers import GlobalMaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input

import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt
import tensorflow.keras
import tensorflow as tf

In [166]:
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification

In [167]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch import LongTensor


In [168]:
from __future__ import print_function
import torch

In [169]:
directory = '../data/protechn_corpus_eval/train'
testDirectory = '../data/protechn_corpus_eval/test'
devDirectory = '../data/protechn_corpus_eval/dev'

In [170]:
import glob
import os, sys

In [76]:
torch.cuda.get_device_name(0)

'Quadro P5000'

In [77]:
from nltk.corpus.reader import wordnet as wordnet

In [78]:
from transformers import AutoTokenizer
import nlpaug.augmenter.word as naw

import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /u/26/nguyenl21/unix/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [79]:
from transformers import BertTokenizer, BertModel
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [80]:
from textblob import TextBlob
import nltk

In [81]:
# config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 16 } ) 
# sess = tf.Session(config=config) 
# keras.backend.set_session(sess)

In [82]:
import re
from spacy.lang.en.stop_words import STOP_WORDS
from nltk.stem.porter import PorterStemmer
import string

def preprocess_text(text, remove_stop = True, stem_words = False, remove_mentions_hashtags = True):
  """
  eg:
  input: preprocess_text("@water #dream hi hello where are you going be there tomorrow happening happen happens",  
  stem_words = True) 
  output: ['tomorrow', 'happen', 'go', 'hello']
  """
  

  if remove_mentions_hashtags:
      text = re.sub(r"@(\w+)", " ", text)
      text = re.sub(r"#(\w+)", " ", text)
      
  text = re.sub(r"[^\x00-\x7F]+", " ", text)
  regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]') # remove punctuation and numbers
  nopunct = regex.sub(" ", text.lower())
  words = (''.join(nopunct)).split()
  
  if(remove_stop):
      words = [w for w in words if w not in STOP_WORDS]
      words = [w for w in words if len(w) > 2]  # remove a,an,of etc.
      
  if(stem_words):
      stemmer = PorterStemmer()
      words = [stemmer.stem(w) for w in words]
      
  return list(words)

New preprocessor

In [83]:
import pathlib
from pathlib import Path
import numpy as np

def read_data(directory):
    ids = []
    texts = []
    labels = []
    for f in directory.glob('*.txt'):
        id = f.name.replace('article', '').replace('.txt','')
        ids.append(id)
        texts.append(f.read_text())
        labels.append(parse_label(f.as_posix().replace('.txt', '.labels.tsv')))
    # labels can be empty 
    return ids, texts, labels

def parse_label(label_path):
    labels = []
    f= Path(label_path)
    
    if not f.exists():
        return labels

    for line in open(label_path):
        parts = line.strip().split('\t')
        # print("parts", parts, int(parts[2]), int(parts[3]))
        labels.append([int(parts[2]), int(parts[3]), parts[1], 0, 0])
    labels = sorted(labels) 

    if labels:
        length = max([label[1] for label in labels]) 
        visit = np.zeros(length)
        res = []
        for label in labels:
            if sum(visit[label[0]:label[1]]):
                label[3] = 1
            else:
               visit[label[0]:label[1]] = 1
            res.append(label)
        return res 
    else:
        return labels

def clean_text(articles, ids):
    texts = []
    for article, id in zip(articles, ids):
        sentences = article.split('\n')
        start = 0
        end = -1
        res = []
        for sentence in sentences:
           start = end + 1
           end = start + len(sentence)  # length of sequence 
           if sentence != "": # if not empty line
               res.append([id, sentence, start, end])
        texts.append(res)
    return texts


In [84]:
context_aug = naw.ContextualWordEmbsAug(device="cuda")
syn_aug = naw.SynonymAug()

In [85]:
syn_aug.augment("This is a big hat satan")

'This comprise a big hat satan'

In [86]:
def is_rare_class(label): 
  return label == "Bandwagon" or label == "Obfuscation,Intentional_Vagueness,Confusion" or label == "Straw_Men" or  label == "Whataboutism" or label == "Red_Herring" 

In [88]:
def get_context(article, span, mode='sentence'):
  article = id2art[article]
  def get_num_words(sentence):
    return len(sentence.split(' '))
  if mode == "title":
    return article.split('\n')[0]
  if mode == "sentence":
    WORD_LEN_LIMIT = 120
    li = span[0]
    ri = span[1]
    span_text = article[li: ri]
    num_words = get_num_words(span_text)
    if num_words >= WORD_LEN_LIMIT:
      return span_text
    remaining_len = WORD_LEN_LIMIT - num_words
    lhs_words = remaining_len // 2
    rhs_words = remaining_len - lhs_words
    li -= 1
    lcount = 0
    while li >= 0 and article[li-1] != '\n' and lcount < lhs_words:
      if article[li] == ' ':
        lcount += 1
      li -= 1
    ri += 1
    rcount = 0
    while ri < len(article) and article[ri] != '\n' and rcount < rhs_words:
      if article[ri] == ' ':
        rcount += 1
      ri += 1
    return article[li+1: ri - 1] 

  return

In [89]:
import pathlib
from pathlib import Path
import numpy as np
from pytorch_pretrained_bert import BertTokenizer

def read_data(directory):
    ids = []
    texts = []
    labels = []
    for f in directory.glob('*.txt'):
        id = f.name.replace('article', '').replace('.txt','')
        ids.append(id)
        texts.append(f.read_text())
        labels.append(parse_label(f.as_posix().replace('.txt', '.labels.tsv')))
    # labels can be empty 
    return ids, texts, labels

def parse_label(label_path):
    labels = []
    f= Path(label_path)
    
    if not f.exists():
        return labels

    for line in open(label_path):
        parts = line.strip().split('\t')
        labels.append([int(parts[2]), int(parts[3]), parts[1], 0, 0])
    labels = sorted(labels) 

    if labels:
        length = max([label[1] for label in labels]) 
        visit = np.zeros(length)
        res = []
        for label in labels:
            if sum(visit[label[0]:label[1]]):
                label[3] = 1
            else:
               visit[label[0]:label[1]] = 1
            res.append(label)
        return res 
    else:
        return labels

def clean_text(articles, ids):
    texts = []
    for article, id in zip(articles, ids):
        sentences = article.split('\n')
        start = 0
        end = -1
        res = []
        for sentence in sentences:
           start = end + 1
           end = start + len(sentence)  # length of sequence 
           if sentence != "": # if not empty line
               res.append([id, sentence, start, end])
        texts.append(res)
    return texts

def make_dataset(directory):
    ids, texts, labels = read_data(directory)
    texts = clean_text(texts, ids)
    res = []
    for text, label in zip(texts, labels):
        # making positive examples
        tmp = [] 
        pos_ind = [0] * len(text)
        for l in label:
            for i, sen in enumerate(text):
                if l[0] >= sen[2] and l[0] < sen[3] and l[1] > sen[3]:
                    l[4] = 1
                    tmp.append(sen + [l[0], sen[3], l[2], l[3], l[4]])
                    pos_ind[i] = 1
                    l[0] = sen[3] + 1
                elif l[0] != l[1] and l[0] >= sen[2] and l[0] < sen[3] and l[1] <= sen[3]: 
                    tmp.append(sen + l)
                    pos_ind[i] = 1
                    
        # making negative examples
        dummy = [0, 0, 'O', 0, 0]
        for k, sen in enumerate(text):
            if pos_ind[k] != 1:
                tmp.append(sen+dummy)
        res.append(tmp)     
    return res


In [90]:
def make_dset(path):
    path_ = Path(path)
    a = make_dataset(path_)
    df_1 = pd.DataFrame(columns=['id','full_sent','start_sent','end_sent','start_prop','end_prop','prop','??','???'])
    for dm in a:
        df_t = pd.DataFrame(dm,columns =['id','full_sent','start_sent','end_sent','start_prop','end_prop','prop','??','???'] )
        df_1 = df_1.append(df_t,ignore_index= True)
    return df_1.iloc[:,:-2]

In [98]:
# ADD only
def make_train_dataset(directory):
  # This is mostly from the original preprocess.py file in their original code
  # However, I removed some redundant data and do preprocessing
    ids, orgtexts, labels = read_data(directory)
    texts = clean_text(orgtexts, ids)
    res = []
    prev_sentence = ""
    prev_major_sentence = ""
    for text, label, oText in zip(texts, labels, orgtexts):
        # making positive examples
        tmp = [] 
        pos_ind = [0] * len(text)
        for l in label:
            for i, sen in enumerate(text):
                if l[0] >= sen[2] and l[0] < sen[3] and l[1] > sen[3]:
                    l[4] = 1
                    sentence = sen[1]
                    pro_sentence = " ".join(preprocess_text(sentence))
                    tmp.append([sen[0],pro_sentence]  + [l[2]])
                    pos_ind[i] = 1
                    l[0] = sen[3] + 1

                elif l[0] != l[1] and l[0] >= sen[2] and l[0] < sen[3] and l[1] <= sen[3]:
                    sentence = (sen + l)[1]
                    pro_sentence = " ".join(preprocess_text((sen + l)[1]))
                    tmp.append([(sen + l)[0], pro_sentence , (sen + l)[6]])
                    pos_ind[i] = 1
#                     fake_text = " ".join(preprocess_text(context_aug.augment(sentence)))
#                     tmp.append(["aug"+(sen + l)[0], fake_text , (sen + l)[6]])  
                    fake_text_add = pro_sentence + " " +prev_major_sentence
                    tmp.append(["add"+(sen + l)[0], fake_text_add , (sen + l)[6]]) 
                    if is_rare_class((sen + l)[6]):
                        fake_text = " ".join(preprocess_text(syn_aug.augment(sentence)))
                        tmp.append(["aug"+(sen + l)[0], fake_text , (sen + l)[6]])  
        # making negative examples
        for k, sen in enumerate(text):
            if pos_ind[k] != 1:
                prev_major_sentence = " ".join(preprocess_text((sen + l)[1]))
                tmp.append([(sen + l)[0], " ".join(preprocess_text((sen + l)[1])), "O"])     
        res.extend(tmp)         
    return res

In [99]:
def make_test_dataset(directory):
    # This is mostly from the original preprocess.py file in their original code
  # However, I removed some redundant data and do preprocessing
    ids, orgtexts, labels = read_data(directory)
    texts = clean_text(orgtexts, ids)
    res = []
    prev_sentence = ""
   
    for text, label, oText in zip(texts, labels, orgtexts):
        # making positive examples
        tmp = [] 
        pos_ind = [0] * len(text)
        for l in label:
            for i, sen in enumerate(text):
                if l[0] >= sen[2] and l[0] < sen[3] and l[1] > sen[3]:
                    l[4] = 1
                    sentence = sen[1]
                    pro_sentence = " ".join(preprocess_text(sentence))
                    tmp.append([sen[0],pro_sentence]  + [l[2]])
                    pos_ind[i] = 1
                    l[0] = sen[3] + 1

                elif l[0] != l[1] and l[0] >= sen[2] and l[0] < sen[3] and l[1] <= sen[3]:
                    sentence = (sen + l)[1]
                    pro_sentence = " ".join(preprocess_text((sen + l)[1]))
                    tmp.append([(sen + l)[0], pro_sentence , (sen + l)[6]])
                    pos_ind[i] = 1
                    
        # making negative examples
        for k, sen in enumerate(text):
            if pos_ind[k] != 1:
                tmp.append([(sen + l)[0], " ".join(preprocess_text((sen + l)[1])), "O"])      
        res.extend(tmp) 
    return res


In [100]:
label2idx = {'Name_Calling,Labeling': 0,
 'Doubt': 1,
 'Whataboutism': 2,
 'Loaded_Language': 3,
 'Straw_Men': 4,
 'Causal_Oversimplification': 5,
 'Exaggeration,Minimisation': 6,
 'Repetition': 7,
 'Red_Herring': 8,
 'Bandwagon': 9,
 'Black-and-White_Fallacy': 10,
 'Slogans': 11,
 'O': 12,
 'Appeal_to_Authority': 13,
 'Obfuscation,Intentional_Vagueness,Confusion': 14,
 'Appeal_to_fear-prejudice': 15,
 'Thought-terminating_Cliches': 16,
 'Reductio_ad_hitlerum': 17,
 'Flag-Waving': 18}
label2idx

{'Name_Calling,Labeling': 0,
 'Doubt': 1,
 'Whataboutism': 2,
 'Loaded_Language': 3,
 'Straw_Men': 4,
 'Causal_Oversimplification': 5,
 'Exaggeration,Minimisation': 6,
 'Repetition': 7,
 'Red_Herring': 8,
 'Bandwagon': 9,
 'Black-and-White_Fallacy': 10,
 'Slogans': 11,
 'O': 12,
 'Appeal_to_Authority': 13,
 'Obfuscation,Intentional_Vagueness,Confusion': 14,
 'Appeal_to_fear-prejudice': 15,
 'Thought-terminating_Cliches': 16,
 'Reductio_ad_hitlerum': 17,
 'Flag-Waving': 18}

In [101]:
labelSize = np.zeros(19)

In [102]:
# Combine to multi-labels and 
def make_multilabels_dataset(data):
    index = -1
    collection = {}
    
    for ID, sentence, label in data:
        label = label2idx[label]
        labelSize[label]+=1
        sentence = sentence.strip()
        if sentence in collection:
            if label not in collection[sentence]:
                collection[sentence].append(label)
            else:
                continue
        else:
#             collection.append([ID, sentence,  [label]])
            collection[sentence] = [label]
            index+=1
#         prev_sentence = sentence
    
    data_processed = np.array(list(collection.items()))
    return data_processed

In [103]:
def make_sentiment_analysis(data):
    collection = []
    sid = SentimentIntensityAnalyzer()
    for sentence, label in data:
#         sentiment_neg_score =  sid.polarity_scores(sentence)['neg']
#         collection.append([sentence,sentiment_neg_score, label])
        sentence_tb = TextBlob(sentence)
        polarity_score = sentence_tb.sentiment.polarity
        subjective_score = sentence_tb.sentiment.polarity

        collection.append([sentence, polarity_score, subjective_score, label])
    return collection

In [97]:
# dev_directory = pathlib.Path(devDirectory)

# dev_data= make_dev_dataset(dev_directory)
# dev_df = pd.DataFrame(dev_data, columns=["Id", "Sentence", "Label"])
# X_dev = dev_df[["Sentence"]]
# Y_dev = dev_df[["Label"]]


In [171]:
tuan_org_data = make_dset(train_directory)

In [30]:
# test_directory = pathlib.Path(testDirectory)
# tuan_test_data = make_dset(test_directory)

In [172]:
tuan_org_data.head()

,id,full_sent,start_sent,end_sent,start_prop,end_prop,prop
0,701225819,"David Duke, the white supremacist icon and for...",61,207,77,99,"Name_Calling,Labeling"
1,701225819,"David Duke, the white supremacist icon and for...",61,207,111,143,"Name_Calling,Labeling"
2,701225819,"David Duke, the white supremacist icon and for...",61,207,131,143,Appeal_to_fear-prejudice
3,701225819,"David Duke, the white supremacist icon and for...",61,207,177,187,"Name_Calling,Labeling"
4,701225819,"However, one individual who represents the Mus...",208,382,305,313,Loaded_Language


In [177]:
import glob
import os
train_direct = glob.glob('../data/protechn_corpus_eval/train/*.txt')
articles = []
def read_articles():
  for filename in train_direct:
    myfile = open(filename)
    article = myfile.read()
    articles.append(article)
    myfile.close()
  article_ids = []
  for filename in train_direct:
#     article_ids.append(filename[41:-4])
    article_ids.append(filename[42:-4])
  return articles, article_ids
articles,art_ids = read_articles()
id2art ={i:a for a,i in zip(articles,art_ids)}

In [105]:
# tuan_test_data = tuan_test_data[(tuan_test_data["full_sent"].str.len() > 0)]
# tuan_test_data

NameError: name 'tuan_test_data' is not defined

In [179]:
spans = [(i,k,j) for i,k,j in zip(tuan_org_data.id,tuan_org_data.start_sent,tuan_org_data.end_sent)]
# len(spans_2)
tuan_org_data['context'] = [get_context(i,(s,e)) for i,s,e in spans]


In [174]:
# tuan_test_data

In [185]:
# tuan_test_data.to_csv("tuan_test_data.csv", index=False)
for i in range(5):
    print(context_aug.augment("I am an apple with wicked hat"))

i am a aunt with wicked hat
i am an onion with the hat
i ate an apple with gold hat
iii am an apple with wicked people
i know an apple with no hat


In [187]:
new_arr = []
i = 0
for art_id, full_sent, start_sent, end_sent, start_prop, end_prop, prop, context in tuan_org_data.to_numpy():
    i += 1

    prev_maj = []
    if prop != "O":
        for i in range(5):
            new_id = "aug"+art_id
            fake_sent = context_aug.augment(full_sent)
            new_arr.append([new_id, fake_sent, start_sent, end_sent, start_prop, end_prop, prop, context])
        for item in prev_maj:
            new_id_add = "add"+art_id
            fake_sent_add = full_sent + item
            new_arr.append([new_id_add, fake_sent_add, start_sent, end_sent, start_prop, end_prop, prop, context])
    if is_rare_class(prop):
        new_id = "aug"+art_id
        fake_sent = context_aug.augment(full_sent)
        new_arr.append([new_id, fake_sent, start_sent, end_sent, start_prop, end_prop, prop, context])
    else:
        if len(prev_maj) < 6:
            prev_maj.append(full_sent)
        else:
            prev_maj[i%6] = full_sent
    new_arr.append([art_id, full_sent, start_sent, end_sent, start_prop, end_prop, prop, context])

In [188]:
pd.DataFrame(new_arr, columns=["art_id", "full_sent", "start_sent", "end_sent", "start_prop", "end_prop", "prop", "context"]).to_csv("ADD5_Context5.csv", index=False)
# 

In [39]:
# pd.DataFrame(new_arr, columns=["art_id", "full_sent", "start_sent", "end_sent", "start_prop", "end_prop", "prop", "context"])

In [ ]:
pd.DataFrame(org_data, columns=["ID","Sentence", "Label"]).to_csv("ADD_Context.csv", index=False)


In [106]:
train_directory = pathlib.Path(directory)
org_data = make_train_dataset(train_directory)

In [107]:
train_data = make_multilabels_dataset(org_data)

In [108]:
train_data_senti = make_sentiment_analysis(train_data)

In [109]:
train_df = pd.DataFrame(train_data_senti, columns=[ "Sentence", "Polarity", "Subjective", "Label"])

train_df2= train_df[train_df['Sentence'].str.len() > 0]

X_train = train_df2[["Sentence"]]
X_train_pol = train_df2[["Polarity"]]
X_train_sub = train_df2[["Subjective"]]
Y_train = train_df2[["Label"]]
Y_train_org = pd.DataFrame(org_data, columns=["ID","Sentence", "Label"])[["Label"]]

In [110]:
labelInfo = pd.DataFrame(org_data, columns=["ID","Sentence", "Label"]).groupby(by=["Label"]).count().reset_index()
labelInfo

,Label,ID,Sentence
0,Appeal_to_Authority,207,207
1,Appeal_to_fear-prejudice,397,397
2,Bandwagon,31,31
3,Black-and-White_Fallacy,200,200
4,Causal_Oversimplification,352,352
5,Doubt,914,914
6,"Exaggeration,Minimisation",787,787
7,Flag-Waving,407,407
8,Loaded_Language,3596,3596
9,"Name_Calling,Labeling",1859,1859


In [114]:
train_df2.head()

,Sentence,Polarity,Subjective,Label
0,david duke white supremacist icon grand wizard...,0.2500,0.2500,"[0, 15]"
1,individual represents muslim community south f...,0.0000,0.0000,[3]
2,videos duke official website davidduke com ref...,-0.0625,-0.0625,[0]
3,rally white nationalist plowed car group peopl...,0.0000,0.0000,[15]
4,february zakkout publicized facebook page biza...,0.4000,0.4000,[15]


In [115]:
test_directory = pathlib.Path(testDirectory)

test_data= make_multilabels_dataset(make_test_dataset(test_directory))

test_data_senti = make_sentiment_analysis(test_data)

test_df = pd.DataFrame(test_data_senti, columns=["Sentence", "Polarity", "Subjective", "Label"])
test_df2 = test_df[(test_df["Sentence"].str.len() > 0)]
X_test = test_df2[["Sentence"]]
X_test_pol = test_df2[["Polarity"]]
X_test_sub = test_df2[["Subjective"]]
Y_test = test_df2[["Label"]]

In [116]:
# make_test_dataset(test_directory)

In [117]:
test_df2.head()

,Sentence,Polarity,Subjective,Label
0,madagascar outbreak inevitable plague resistan...,0.000000,0.000000,[13]
1,newest warning outbreak airborne pneumonic pla...,-0.166667,-0.166667,[0]
2,officials warn inevitable bacterial infection ...,0.000000,0.000000,"[13, 7]"
3,experts warn treat infection antibiotics resis...,-0.500000,-0.500000,"[13, 7, 15]"
4,according daily mail madagascar healthcare sys...,-0.233333,-0.233333,[13]


In [118]:
# convert text to vector
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(X_train.Sentence.values)

# X_train_tok1 = tokenizer.texts_to_sequences(X_train.Sentence.values)
# X_test_tok1 = tokenizer.texts_to_sequences(X_test.Sentence.values)

In [119]:

# vocab_size = len(tokenizer.word_index) + 1
# maxlen = int(max([len(i) for i in X_train_tok1]))
# # maxlen = 14

# X_train_tok1 = pad_sequences(X_train_tok1, padding='post', maxlen=maxlen)
# X_test_tok1 = pad_sequences(X_test_tok1, padding='post', maxlen=maxlen)

In [120]:
for x in X_test.Sentence.values:
    if x == "":
        print("ahoy", x)

In [121]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

I1125 23:53:57.515397 139915186120512 file_utils.py:196] https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache, downloading to /tmp/tmp_41blfv5
100%|██████████| 231508/231508 [00:00<00:00, 493800.10B/s]
I1125 23:53:58.504562 139915186120512 file_utils.py:209] copying /tmp/tmp_41blfv5 to cache at /u/26/nguyenl21/unix/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
I1125 23:53:58.512594 139915186120512 file_utils.py:213] creating metadata file for /u/26/nguyenl21/unix/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
I1125 23:53:58.516593 139915186120512 file_utils.py:219] removing temp file /tmp/tmp_41blfv5
I1125 23:53:58.517664 139915186120512 tokenization.py:128] loading vocabulary file https://s3.amazonaws.com/models.huggingface.co

In [122]:

# Tokenize our sentence with the BERT tokenizer.
# tokenized_text = 
# X_train_tok = 
# (X_train.Sentence.values)
X_train_tok = list(filter(lambda x: len(x), [bert_tokenizer.convert_tokens_to_ids(bert_tokenizer.tokenize(x)) for x in X_train.Sentence.values]))
X_test_tok = list(filter(lambda x: len(x), [bert_tokenizer.convert_tokens_to_ids(bert_tokenizer.tokenize(x)) for x in X_test.Sentence.values]))


In [123]:
seq_lengths = LongTensor(list([len(i) for i in X_train_tok]))
test_seq_lengths = LongTensor(list([len(i) for i in X_test_tok]))

In [124]:
# from keras.preprocessing.sequence import pad_sequences

# vocab_size = len(tokenizer.word_index) + 1
maxlen = 14
# int(max([len(i) for i in X_train_tok]))


vocab_size = 30000
X_train_tok = pad_sequences(X_train_tok, padding='post', maxlen=maxlen)
X_test_tok = pad_sequences(X_test_tok, padding='post', maxlen=maxlen)

maxlen

14

In [125]:
seq_lengths, perm_idx = seq_lengths.sort(0, descending=True)
test_seq_lengths, test_perm_idx = test_seq_lengths.sort(0, descending=True)

In [126]:
perm_idx.shape

torch.Size([15990])

In [127]:
X_train_tok = X_train_tok[perm_idx]
X_test_tok = X_test_tok[test_perm_idx]

In [128]:
X_train_tok[1]

array([ 6313,  7401,  7327,  7507, 15061,  8656, 16873,  4490,  7842,
       14194,  3771, 11060,  2204,  3114], dtype=int32)

In [129]:
# y_size = Y_train.groupby(["Label"]).size()
# nSamples = [0]*19
# for key in label2idx:
#     nSamples[label2idx[key]] = y_size[key]
# nSamples

In [130]:
# sumSamples = sum(nSamples)



In [131]:
# weights = [1 / (x / sumSamples) for x in nSamples]
# weights

In [132]:
# weights = torch.FloatTensor(weights).cuda()
# weights = torch.FloatTensor(weights)

In [133]:
np.array(list(label2idx.items()))

array([['Name_Calling,Labeling', '0'],
       ['Doubt', '1'],
       ['Whataboutism', '2'],
       ['Loaded_Language', '3'],
       ['Straw_Men', '4'],
       ['Causal_Oversimplification', '5'],
       ['Exaggeration,Minimisation', '6'],
       ['Repetition', '7'],
       ['Red_Herring', '8'],
       ['Bandwagon', '9'],
       ['Black-and-White_Fallacy', '10'],
       ['Slogans', '11'],
       ['O', '12'],
       ['Appeal_to_Authority', '13'],
       ['Obfuscation,Intentional_Vagueness,Confusion', '14'],
       ['Appeal_to_fear-prejudice', '15'],
       ['Thought-terminating_Cliches', '16'],
       ['Reductio_ad_hitlerum', '17'],
       ['Flag-Waving', '18']], dtype='<U43')

In [134]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
Y_train_bi = mlb.fit_transform(Y_train["Label"])
Y_train_df = pd.DataFrame(mlb.fit_transform(Y_train["Label"]),columns=mlb.classes_)
Y_train_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [135]:
Y_test_df = pd.DataFrame(mlb.fit_transform(Y_test["Label"]),columns=mlb.classes_)

In [136]:
Y_train_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
5,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
6,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
7,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
9,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [137]:
# 0th output
y0_train = Y_train_df[[0]].values
y0_test =  Y_test_df[[0]].values

# First output
y1_train = Y_train_df[[1]].values
y1_test =  Y_test_df[[1]].values

# Second output
y2_train = Y_train_df[[2]].values
y2_test =  Y_test_df[[2]].values

# Third output
y3_train = Y_train_df[[3]].values
y3_test =  Y_test_df[[3]].values

# Fourth output
y4_train = Y_train_df[[4]].values
y4_test =  Y_test_df[[4]].values

# Fifth output
y5_train = Y_train_df[[5]].values
y5_test =  Y_test_df[[5]].values

# Sixth output
y6_train = Y_train_df[[6]].values
y6_test =  Y_test_df[[6]].values

y7_train = Y_train_df[[7]].values
y7_test =  Y_test_df[[7]].values

y8_train = Y_train_df[[8]].values
y8_test =  Y_test_df[[8]].values

y9_train = Y_train_df[[9]].values
y9_test =  Y_test_df[[9]].values

y10_train = Y_train_df[[10]].values
y10_test =  Y_test_df[[10]].values

y11_train = Y_train_df[[11]].values
y11_test =  Y_test_df[[11]].values

y12_train = Y_train_df[[12]].values
y12_test =  Y_test_df[[12]].values

y13_train = Y_train_df[[13]].values
y13_test =  Y_test_df[[13]].values

y14_train = Y_train_df[[14]].values
y14_test =  Y_test_df[[14]].values

y15_train = Y_train_df[[15]].values
y15_test =  Y_test_df[[15]].values

y16_train = Y_train_df[[16]].values
y16_test =  Y_test_df[[16]].values

y17_train = Y_train_df[[17]].values
y17_test =  Y_test_df[[17]].values

y18_train = Y_train_df[[18]].values
y18_test =  Y_test_df[[18]].values


In [138]:
train_output = [y0_train, y1_train, y2_train, y3_train, y4_train, y5_train, y6_train, y7_train,
               y8_train, y9_train, y10_train, y11_train, y12_train, y13_train, y14_train, 
               y15_train, y16_train, y17_train, y18_train]

# train_output = [y0_train]

In [139]:
len(X_test_tok)

3423

In [140]:
from keras import backend as K 
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.
W1125 23:54:27.052460 139915186120512 deprecation_wrapper.py:119] From /work/modules/Ubuntu/14.04/amd64/common/anaconda3/20190819-gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W1125 23:54:27.053715 139915186120512 deprecation_wrapper.py:119] From /work/modules/Ubuntu/14.04/amd64/common/anaconda3/20190819-gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W1125 23:54:27.054872 139915186120512 deprecation_wrapper.py:119] From /work/modules/Ubuntu/14.04/amd64/common/anaconda3/20190819-gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:186: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W1125 23:54:28.506495 139915186120512 deprecation_wrapper.py:119] From /work/modules/Ubuntu/14.04/amd6

['/job:localhost/replica:0/task:0/device:GPU:0']

In [141]:
vocab_size

30000

In [142]:
# bn = BatchNormalization()(input_1)
# Embedding(vocab_size, 100,  trainable=True)(bn)
sum(labelSize)

29718.0

In [143]:
total = sum(labelSize)
weights = [1 / (x / total) for x in labelSize]
weights

[13.051383399209486,
 27.214285714285715,
 139.5211267605634,
 6.651298119964189,
 825.5,
 71.09569377990431,
 30.17055837563452,
 22.77241379310345,
 330.2,
 762.0,
 117.92857142857142,
 96.80130293159608,
 1.8190610271163614,
 93.15987460815047,
 724.829268292683,
 45.23287671232877,
 166.95505617977528,
 283.0285714285714,
 49.61268781302171]

In [144]:
X_train_tok[1]

array([ 6313,  7401,  7327,  7507, 15061,  8656, 16873,  4490,  7842,
       14194,  3771, 11060,  2204,  3114], dtype=int32)

In [145]:
K.clear_session()

input_1 = Input(shape=(maxlen,))

polarity_input = Input(shape=(1,))
subjective_input = Input(shape=(1,))

embedding_layer = Embedding(vocab_size, 100,  trainable=True)(input_1)

LSTM_org = Bidirectional(LSTM(256, dropout=0.2, recurrent_dropout=0.2))(embedding_layer)
sentiment_input = tf.keras.layers.Concatenate(axis=1)([polarity_input, subjective_input]) 
LSTM_Layer1 = tf.keras.layers.Concatenate(axis=1)([LSTM_org, sentiment_input]) 

# BatchNormalization()(lol)

output0 = Dense(1, activation='sigmoid')(LSTM_Layer1)

output1 = Dense(1, activation='sigmoid')(LSTM_Layer1)
# output1 = BatchNormalization()(output1)
output2 = Dense(1, activation='sigmoid')(LSTM_Layer1)
output3 = Dense(1, activation='sigmoid')(LSTM_Layer1)
output4 = Dense(1, activation='sigmoid')(LSTM_Layer1)
output5 = Dense(1, activation='sigmoid')(LSTM_Layer1)
output6 = Dense(1, activation='sigmoid')(LSTM_Layer1)

output7 = Dense(1, activation='sigmoid')(LSTM_Layer1)
output8 = Dense(1, activation='sigmoid')(LSTM_Layer1)
output9 = Dense(1, activation='sigmoid')(LSTM_Layer1)
output10 = Dense(1, activation='sigmoid')(LSTM_Layer1)
output11 = Dense(1, activation='sigmoid')(LSTM_Layer1)
output12 = Dense(1, activation='sigmoid')(LSTM_Layer1)

output13 = Dense(1, activation='sigmoid')(LSTM_Layer1)
output14 = Dense(1, activation='sigmoid')(LSTM_Layer1)
output15 = Dense(1, activation='sigmoid')(LSTM_Layer1)
output16 = Dense(1, activation='sigmoid')(LSTM_Layer1)
output17 = Dense(1, activation='sigmoid')(LSTM_Layer1)
output18 = Dense(1, activation='sigmoid')(LSTM_Layer1)

model = Model(inputs=[input_1, polarity_input, subjective_input], outputs=[output0, output1, output2, output3, output4, output5, output6, output7, output8, output9, output10, output11, output12, output13, output14, output15, output16, output17, output18])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'],
#     loss_weights=weights
)

W1125 23:54:39.216851 139915186120512 deprecation_wrapper.py:119] From /work/modules/Ubuntu/14.04/amd64/common/anaconda3/20190819-gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W1125 23:54:39.250687 139915186120512 deprecation.py:506] From /work/modules/Ubuntu/14.04/amd64/common/anaconda3/20190819-gpu/lib/python3.7/site-packages/tensorflow/python/keras/initializers.py:119: calling RandomUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1125 23:54:39.270962 139915186120512 deprecation.py:506] From /work/modules/Ubuntu/14.04/amd64/common/anaconda3/20190819-gpu/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.p

In [146]:
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 14)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 14, 100)      3000000     input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 1)]          0                                            
______________________________________________________________________________________________

In [147]:
X_train_tok

array([[ 3819,  2819,  2512, ...,  6633, 24759,  2906],
       [ 6313,  7401,  7327, ..., 11060,  2204,  3114],
       [10696,  2094,  4490, ...,  8656, 16873,  4490],
       ...,
       [ 4752,     0,     0, ...,     0,     0,     0],
       [ 5284,     0,     0, ...,     0,     0,     0],
       [23156,     0,     0, ...,     0,     0,     0]], dtype=int32)

In [148]:
history = model.fit(x=[X_train_tok, X_train_pol.values, X_train_sub.values], y=[y0_train, y1_train, y2_train, y3_train, y4_train, y5_train, y6_train, y7_train, y8_train, y9_train, y10_train, y11_train, y12_train, y13_train, y14_train, y15_train, y16_train, y17_train, y18_train], batch_size=64, epochs=10, verbose=1)


Epoch 1/10
15990/15990 [==============================] - 19s 1ms/sample - loss: 3.2010 - dense_loss: 0.3414 - dense_1_loss: 0.2334 - dense_2_loss: 0.0749 - dense_3_loss: 0.4997 - dense_4_loss: 0.0353 - dense_5_loss: 0.1251 - dense_6_loss: 0.2058 - dense_7_loss: 0.2217 - dense_8_loss: 0.0461 - dense_9_loss: 0.0366 - dense_10_loss: 0.0841 - dense_11_loss: 0.0949 - dense_12_loss: 0.6935 - dense_13_loss: 0.0885 - dense_14_loss: 0.0360 - dense_15_loss: 0.1277 - dense_16_loss: 0.0679 - dense_17_loss: 0.0545 - dense_18_loss: 0.1338 - dense_acc: 0.8967 - dense_1_acc: 0.9385 - dense_2_acc: 0.9870 - dense_3_acc: 0.8082 - dense_4_acc: 0.9962 - dense_5_acc: 0.9752 - dense_6_acc: 0.9507 - dense_7_acc: 0.9460 - dense_8_acc: 0.9943 - dense_9_acc: 0.9962 - dense_10_acc: 0.9848 - dense_11_acc: 0.9819 - dense_12_acc: 0.5364 - dense_13_acc: 0.9849 - dense_14_acc: 0.9934 - dense_15_acc: 0.9733 - dense_16_acc: 0.9883 - dense_17_acc: 0.9901 - dense_18_acc: 0.9739
Epoch 2/10
15990/15990 [===================

In [149]:
score = model.evaluate(x=[X_test_tok, X_test_pol.values, X_test_sub.values], y=[y0_test, y1_test, y2_test, y3_test, y4_test, y5_test, y6_test, y7_test, y8_test, y9_test, y10_test, y11_test, y12_test, y13_test, y14_test, y15_test, y16_test, y17_test, y18_test], verbose=1)

print("test loss, test precision:", score)

3423/3423 [==============================] - 3s 1ms/sample - loss: 4.2067 - dense_loss: 0.3652 - dense_1_loss: 0.1975 - dense_2_loss: 0.0520 - dense_3_loss: 0.7079 - dense_4_loss: 0.0062 - dense_5_loss: 0.0706 - dense_6_loss: 0.1666 - dense_7_loss: 0.2908 - dense_8_loss: 0.0321 - dense_9_loss: 0.0104 - dense_10_loss: 0.0544 - dense_11_loss: 0.0661 - dense_12_loss: 1.6178 - dense_13_loss: 0.0983 - dense_14_loss: 0.0143 - dense_15_loss: 0.2309 - dense_16_loss: 0.0345 - dense_17_loss: 0.0266 - dense_18_loss: 0.1645 - dense_acc: 0.9027 - dense_1_acc: 0.9530 - dense_2_acc: 0.9927 - dense_3_acc: 0.7642 - dense_4_acc: 0.9994 - dense_5_acc: 0.9904 - dense_6_acc: 0.9600 - dense_7_acc: 0.9384 - dense_8_acc: 0.9959 - dense_9_acc: 0.9988 - dense_10_acc: 0.9927 - dense_11_acc: 0.9907 - dense_12_acc: 0.5004 - dense_13_acc: 0.9854 - dense_14_acc: 0.9982 - dense_15_acc: 0.9609 - dense_16_acc: 0.9947 - dense_17_acc: 0.9968 - dense_18_acc: 0.9737
test loss, test precision: [4.2066764085570725, 0.3651816

In [150]:
test_output = [y0_test, y1_test, y2_test, y3_test, y4_test, y5_test, y6_test, y7_test, y8_test, y9_test, y10_test, y11_test, y12_test, y13_test, y14_test, y15_test, y16_test, y17_test, y18_test]

In [151]:
predictions = model.predict([X_test_tok, X_test_pol.values, X_test_sub.values])



In [152]:
predictions_transposed = np.stack(predictions).transpose(1, 0, 2)
thresholded = np.round(predictions_transposed.reshape(predictions_transposed.shape[0], predictions_transposed.shape[1]))

In [153]:
v = Y_test_df

In [154]:
predictions[12]

array([[0.48646614],
       [0.9261675 ],
       [0.9982908 ],
       ...,
       [0.54804343],
       [0.99833965],
       [0.13589858]], dtype=float32)

In [155]:
from sklearn.metrics import precision_recall_fscore_support as score
precision, recall, fscore, support = score(v, thresholded, average="weighted")

/work/modules/Ubuntu/14.04/amd64/common/anaconda3/20190819-gpu/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.



In [156]:
print('fscore: {}'.format(fscore))
# The support is the number of occurrences of each class in y_true.
precision

fscore: 0.3966267853886318


0.4738369220408294

In [157]:
rounded_predictions = np.round(predictions)

In [160]:
precision_arr = []
recall_arr = []
fscore_arr = []
for i, predict in enumerate(rounded_predictions):
    precision, recall, fscore, support = score(test_output[i], predict)
    precision_arr.append(precision[1])
    recall_arr.append(recall[1])
    fscore_arr.append(fscore[1])

In [163]:
np.average(fscore_arr)

0.04594695358993862

In [118]:
precision_arr

[array([0.94352792, 0.05904059]),
 array([0.9742515 , 0.02409639]),
 array([0.99269647, 0.        ]),
 array([0.89131199, 0.11034483]),
 array([0.99941572, 0.        ]),
 array([0.99031974, 0.        ]),
 array([0.97454115, 0.02222222]),
 array([0.95017794, 0.03921569]),
 array([0.99591002, 0.        ]),
 array([0.99883143, 0.        ]),
 array([0.99269647, 0.        ]),
 array([0.99065148, 0.        ]),
 array([0.28851413, 0.69812856]),
 array([0.98538866, 0.        ]),
 array([0.99824715, 0.        ]),
 array([0.9636896, 0.125    ]),
 array([0.99474145, 0.        ]),
 array([0.99678644, 0.        ]),
 array([0.97445684, 0.05882353])]

In [119]:
# Load additional text
# text_directory = 'newtext.txt'
# os.chdir(text_directory)
f = open("newtext.txt", "r") 
content = f.read()


In [120]:
content

'The onset of the global pandemic in March understandably led to upheaval and delays as people and organizations adjusted to a new, tenuous normal. USCIS, however, simply shut down its services for a period of over two months. For more than 70 days, it did not offer naturalization ceremonies or visa and asylum interviews. Despite being pressed by many advocates who were alarmed at these delays, USCIS refused to transition to virtual platforms to continue operations, citing vague supposed legal limitations. The immigration-services company Boundless estimated that each day without naturalization ceremonies translated into “2,100 potential new voters” being disenfranchised.\n\nSince restarting naturalization ceremonies in June, USCIS has cleared the backlog of soon-to-be-citizens who had their ceremonies delayed this spring, but at the expense of hundreds of thousands more who are waiting for theirs to be scheduled. Meanwhile, backlogs in other areas, including new naturalization applica

In [121]:
from __future__ import unicode_literals, print_function
from spacy.lang.en import English # update
raw_text = content
nlp = English()
nlp.add_pipe(nlp.create_pipe('sentencizer')) # updated
doc = nlp(raw_text)
sentences = [sent.string.strip() for sent in doc.sents]
reshape_data = np.array(sentences).reshape(-1)
reshape_data
new_Data = pd.DataFrame(reshape_data, columns=["Sentence"])
new_Data.to_csv('out.csv', index=False)  

In [122]:
new_Data

,Sentence
0,The onset of the global pandemic in March unde...
1,"USCIS, however, simply shut down its services ..."
2,"For more than 70 days, it did not offer natura..."
3,Despite being pressed by many advocates who we...
4,The immigration-services company Boundless est...
5,Since restarting naturalization ceremonies in ...
6,"Meanwhile, backlogs in other areas, including ..."
7,As Sarah Pierce of the Migration Policy Instit...
8,But thanks to budget shortfalls stemming from ...


In [100]:

# ADD Rhetorical_feature
"""
Parses the argument lexicon by Somasundaran, Ruppenhofer & Wiebe (2007):
http://people.cs.pitt.edu/~wiebe/pubs/papers/sigdial07.pdf
http://mpqa.cs.pitt.edu/lexicons/arg_lexicon/
Span identification task: Encodes whether a given token is contained in a
phrase that matches a rhetorical pattern.
Technique classification: Encodes whether a given fragment contains such a
rhetorically salient phrase.
"""
import re


path = '../data/arglex/'
strategies_5 = ['authority', 'doubt', 'emphasis', 'generalization', 'priority']
strategies_full = ['assessments', 'authority', 'causation', 'conditionals',
                   'contrast', 'difficulty', 'doubt', 'emphasis',
                   'generalization', 'inconsistency',
                   'inyourshoes', 'necessity', 'possibility', 'priority',
                   'rhetoricalquestion', 'structure', 'wants']
macros = ['modals', 'spoken', 'wordclasses', 'pronoun', 'intensifiers']
# macro -> list of expansions
expansions = dict()
# strategy -> list of regexes
regexes = dict()


def init(strategies, verbose=False):
    for macro in macros:
        with open(path + macro + '.tff') as f:
            for line in f:
                if line.startswith('#'):
                    # comment
                    continue
                line = line.strip()
                if len(line) == 0:
                    continue
                fields = line.split('=')
                macro_word = fields[0]
                expansion_list = fields[1][1:-1]  # Strip away { and }
                # The lists use both ', ' and ',' as separators.
                expansion_list = expansion_list.replace(', ', ',')
                expansion_list = expansion_list.split(',')
                expansion_list = '(' + '|'.join(expansion_list) + ')'
                expansions[macro_word] = expansion_list

    if verbose:
        print('Macros and their expansions:')
        for m in expansions:
            print(m, expansions[m])
        print()

    for strategy in strategies:
        regexes[strategy] = []
        with open(path + strategy + '.tff') as f:
            for line in f:
                if line.startswith('#'):
                    # comment
                    continue
                line = line.strip()
                if len(line) == 0:
                    continue
                for macro in expansions:
                    line = line.replace('(' + macro + ' )?',
                                        '(' + expansions[macro] + ' )?')
                    line = line.replace('(' + macro + ')', expansions[macro])
                line = line.replace('\\', '')
                regexes[strategy] += ['\\b' + line + '\\b']

    if verbose:
        print('Regexes for rhetorical strategies:')
        for s in regexes:
            print(s, regexes[s])
        print()



def find_rhetorical_strategies(token_list, strategy):
    sentence = ' '.join(token_list)
    token_indices = set()
    if strategy is 'any':
        strats = [s for s in regexes]
    else:
        strats = [strategy]
    for strategy in strats:
        for regex in regexes[strategy]:
            for match in re.finditer(regex, sentence):
                # print(strategy.upper(), '--', match.group(),
                #       '--', match.span(), '--', regex)
                start_idx = match.span()[0]
                end_idx = match.span()[1]
                # idx in the token list
                token_indices.add(sentence[:start_idx].count(' '))
                token_indices.add(sentence[:end_idx].count(' '))
    return token_indices


def parse_input_file_si(infile, outfile, full=True, indiv_cols=False):
    """
    full: if True, use all strategies, if false, use the 5 most important
           strategies. Used for generating the preprocessing description.
           Should match the initialization
    indiv_cols: if True, each rhetorical strategy is represented by its own
                column. If False, matches for any strategy are represented
                in a single joint feature column.
    """
    with open(infile, encoding='utf8') as f_in:
        lines = f_in.readlines()
        lines.append('eof\teof\teof\teof\teof\teof\n')

    if indiv_cols:
        strategies = [s for s in regexes]
    else:
        strategies = ['any']

    with open(outfile, 'w', encoding='utf8') as f_out:
        rows = []
        tokens = []
        prev_article = ''
        first_line = True
        for line in lines:

            # Comments + header
            if line.startswith('#'):
                f_out.write(line)
                continue
            if first_line:
                f_out.write('# rhetorical_features: ArguingLexicon (')
                if full:
                    f_out.write('full, ')
                else:
                    f_out.write('5 main strategies, ')
                if indiv_cols:
                    f_out.write('individual feature columns')
                else:
                    f_out.write('joint feature column')
                f_out.write(')\n')
                first_line = False
                labels = line.strip().split('\t')
                try:
                    doc_idx = labels.index('document_id')
                except ValueError:
                    doc_idx = 0
                try:
                    word_idx = labels.index('token')
                except ValueError:
                    word_idx = 4
                for strategy in strategies:
                    labels.append('arglex_' + strategy)
                f_out.write('\t'.join(labels) + '\n')
                continue

            line = line[:-1]  # Remove \n
            fields = line.split('\t')
            article = fields[doc_idx]
            word = fields[word_idx]

            if article != prev_article:
                for strategy in strategies:
                    indices = find_rhetorical_strategies(tokens, strategy)
                    rows_new = []
                    for i, row in enumerate(rows):
                        if i in indices:
                            rows_new.append(row + '\t1')
                        else:
                            rows_new.append(row + '\t0')
                    rows = rows_new
                for row in rows:
                    f_out.write(row + '\n')
                tokens = []
                rows = []
            tokens.append(word)
            rows.append(line)
            prev_article = article


def annotate_tc(in_file):
    strategies = [s for s in regexes]
    strategies.sort()
    matched_strategies = {}

    with open(in_file, encoding='utf8') as f:
        lines = f.readlines()

    with open(in_file, 'w', encoding='utf8') as f:
        f.write(lines[0].strip() + '\t' + '\t'.join(strategies) + '\n')
        for line in lines[1:]:
            f.write(line.strip())
            text = line.split('\t')[4].strip().lower()
            for strategy in strategies:
                matched = 0
                for regex in regexes[strategy]:
                    for match in re.finditer(regex, text):
                        matched = 1
                        break
                f.write('\t' + str(matched))
                if matched:
                    try:
                        matched_strategies[strategy] += 1
                    except KeyError:
                        matched_strategies[strategy] = 1
            f.write('\n')
    for matched_strat in matched_strategies:
        strategies.remove(matched_strat)
    matched_strategies = sorted(matched_strategies.items(),
                                key=lambda s: s[1], reverse=True)
    for s in matched_strategies:
        print(s)
    print("Strategies without occurrences:", strategies)
    print()


if __name__ == "__main__":
    ### Task 1: Span identification
    init(strategies_full)
    parse_input_file_si('../data/train-improved-sentiwordnet.tsv',
                        '../data/train-improved-sentiwordnet-arguingfullindiv.tsv')
    parse_input_file_si('../data/dev-improved-sentiwordnet.tsv',
                        '../data/dev-improved-sentiwordnet-arguingfullindiv.tsv')
    parse_input_file_si('../data/test-improved-sentiwordnet.tsv',
                        '../data/test-improved-sentiwordnet-arguingfullindiv.tsv')

    ### Task 2: Technique identification
    init(strategies_full)
    # These strategies don't actually appear in the training data:
    regexes.pop('inyourshoes')
    regexes.pop('doubt')
    # These strategies barely appear in the training data (<10 occurrences):
    regexes.pop('difficulty')
    regexes.pop('conditionals')
    regexes.pop('assessments')
    regexes.pop('rhetoricalquestion')
    annotate_tc('../data/tc-train.tsv')
    annotate_tc('../data/tc-dev.tsv')
    annotate_tc('../data/tc-test.tsv')

FileNotFoundError: [Errno 2] No such file or directory: '../data/arglex/modals.tff'

In [1281]:
# !pip install TextBlob